Autor: Esteban Suárez Calvo

In [32]:
from datetime import timedelta
from time import time

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import n_local, z_feature_map, zz_feature_map
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Sampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import (
    COBYLA,
    NELDER_MEAD,
    POWELL,
    SPSA,
    CRS,
    DIRECT_L,
)
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# CCAM - Práctica 2: Redes Neuronales Cuánticas

Antes de empezar, fijaremos la semilla aleatoria, de manera que los resultados obtenidos en el notebook sean reproducibles.

In [2]:
algorithm_globals.random_seed = 1

## 1.Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización MinMax.

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Reducimos la dimensionalidad del dataset utilizando una reducción PCA. De no hacerlo, necesitaríamos un total de 30 qubits, pues tenemos 30 características, y cada una se representa con un qubit. El problema está en que, aunque 30 qubits pueden no parecer demasiados, puede suponer un costo computacional bastante elevado el tener que emularlos.

Sin embargo, reducir la dimensionalidad tiene también su parte negativa, y es que al reducirla, estas dimensiones colapsan en menos, lo que hace que las clases que, anteriormente, podrían ser separables, puedan ya no serlo, por lo que debemos de ser cuidadosos y no pasarnos al reducir las dimensiones.

In [4]:
X = PCA(n_components=12).fit_transform(X)
number_of_features = X.shape[1]
print(f"Nuevo número de características = {number_of_features}")

Nuevo número de características = 12


### Dividir el conjunto entre entrenamiento y test

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.20, random_state=algorithm_globals.random_seed
)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


### Clasificación con un SVM

Para comprobar que el problema es resoluble con la reducción de dimensionalidad que hemos aplicado antes, utilizamos un SVM

In [6]:
svc = SVC()
svc.fit(X_train, Y_train)

train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)

print(f"SVC en entrenamiento: {train_score}")
print(f"SVC en test: {test_score}")

SVC en entrenamiento: 0.9824175824175824
SVC en test: 0.9736842105263158


## Desarrolla una QNN para predecir la clase de cada caso

A continuación, definimos una función que, dado un `feature_map`, un `ansatz` y un `optimizer` arbitrarios, crea el `VQC` correspondiente y lo entrena. Definimos también un callback para hacer más interactivo el proceso de entrenamiento, de manera que, a cada iteración, se muestre el número de la iteración y el *loss*:

In [23]:
def train_VQC(feature_map, ansatz, optimizer) -> VQC:
    global X_train, Y_train

    number_of_features = X.shape[1]
    feature_map = zz_feature_map(number_of_features)

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=MyCallback().print_loss,
        sampler=Sampler(AerSimulator()),
    )

    start = time()
    vqc.fit(X_train, Y_train)
    total_time = time() - start
    print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return vqc


class MyCallback:
    def __init__(self):
        self._loss_history = []

    def print_loss(self, weights, obj_func_eval):
        self._loss_history.append(obj_func_eval)

        iteration = len(self._loss_history)

        print(f"Iteración {iteration} - Loss: {obj_func_eval:.4f}")

Definimos también una función para poder evaluar el clasificador.

In [8]:
def evaluate_VQC(vqc: VQC) -> None:
    global X_train, Y_train

    train_score = vqc.score(X_train, Y_train)
    print(f"SVC en entrenamiento: {train_score}")

    test_score = vqc.score(X_test, Y_test)
    print(f"SVC en test: {test_score}")

Ahora que tenemos una función para crear el `VQC` y para evaluarlo, podemos realizar pruebas con distintos clasificadores y ver que tan bien funcionan.

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=1)
optimizer = COBYLA(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9883
Iteración 2 - Loss: 0.9905
Iteración 3 - Loss: 0.9914
Iteración 4 - Loss: 0.9873
Iteración 5 - Loss: 0.9909
Iteración 6 - Loss: 0.9904
Iteración 7 - Loss: 0.9869
Iteración 8 - Loss: 0.9898
Iteración 9 - Loss: 0.9889
Iteración 10 - Loss: 0.9877
Iteración 11 - Loss: 0.9868
Iteración 12 - Loss: 0.9863
Iteración 13 - Loss: 0.9870
Iteración 14 - Loss: 0.9738
Iteración 15 - Loss: 0.9755
Iteración 16 - Loss: 0.9753
Iteración 17 - Loss: 0.9802
Iteración 18 - Loss: 0.9801
Iteración 19 - Loss: 0.9755
Iteración 20 - Loss: 0.9781
Iteración 21 - Loss: 0.9772
Iteración 22 - Loss: 0.9785
Iteración 23 - Loss: 0.9748
Iteración 24 - Loss: 0.9763
Iteración 25 - Loss: 0.9731
Iteración 26 - Loss: 0.9846
Iteración 27 - Loss: 0.9908
Iteración 28 - Loss: 0.9667
Iteración 29 - Loss: 0.9674
Iteración 30 - Loss: 0.9652
Iteración 31 - Loss: 0.9648
Iteración 32 - Loss: 0.9693
Iteración 33 - Loss: 0.9639
Iteración 34 - Loss: 0.9652
Iteración 35 - Loss: 0.9690
Iteración 36 - Loss: 0.9690
I

In [10]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=2)
optimizer = COBYLA(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9978
Iteración 2 - Loss: 1.0021
Iteración 3 - Loss: 0.9942
Iteración 4 - Loss: 0.9929
Iteración 5 - Loss: 0.9909
Iteración 6 - Loss: 0.9947
Iteración 7 - Loss: 0.9907
Iteración 8 - Loss: 0.9961
Iteración 9 - Loss: 0.9928
Iteración 10 - Loss: 0.9919
Iteración 11 - Loss: 0.9929
Iteración 12 - Loss: 0.9923
Iteración 13 - Loss: 0.9922
Iteración 14 - Loss: 0.9746
Iteración 15 - Loss: 0.9739
Iteración 16 - Loss: 0.9725
Iteración 17 - Loss: 0.9730
Iteración 18 - Loss: 0.9720
Iteración 19 - Loss: 0.9709
Iteración 20 - Loss: 0.9746
Iteración 21 - Loss: 0.9731
Iteración 22 - Loss: 0.9711
Iteración 23 - Loss: 0.9776
Iteración 24 - Loss: 0.9694
Iteración 25 - Loss: 0.9725
Iteración 26 - Loss: 0.9938
Iteración 27 - Loss: 0.9744
Iteración 28 - Loss: 0.9729
Iteración 29 - Loss: 0.9723
Iteración 30 - Loss: 0.9753
Iteración 31 - Loss: 0.9716
Iteración 32 - Loss: 0.9735
Iteración 33 - Loss: 0.9740
Iteración 34 - Loss: 0.9700
Iteración 35 - Loss: 0.9732
Iteración 36 - Loss: 0.9752
I

In [11]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9857
Iteración 2 - Loss: 0.9927
Iteración 3 - Loss: 0.9912
Iteración 4 - Loss: 0.9849
Iteración 5 - Loss: 0.9855
Iteración 6 - Loss: 0.9883
Iteración 7 - Loss: 0.9846
Iteración 8 - Loss: 0.9905
Iteración 9 - Loss: 0.9854
Iteración 10 - Loss: 0.9865
Iteración 11 - Loss: 0.9855
Iteración 12 - Loss: 0.9881
Iteración 13 - Loss: 0.9900
Iteración 14 - Loss: 0.9961
Iteración 15 - Loss: 0.9968
Iteración 16 - Loss: 0.9864
Iteración 17 - Loss: 0.9874
Iteración 18 - Loss: 0.9880
Iteración 19 - Loss: 0.9873
Iteración 20 - Loss: 0.9885
Iteración 21 - Loss: 0.9834
Iteración 22 - Loss: 0.9867
Iteración 23 - Loss: 0.9856
Iteración 24 - Loss: 0.9900
Iteración 25 - Loss: 0.9873
Iteración 26 - Loss: 1.0014
Iteración 27 - Loss: 0.9837
Iteración 28 - Loss: 0.9865
Iteración 29 - Loss: 0.9849
Iteración 30 - Loss: 0.9833
Iteración 31 - Loss: 0.9904
Iteración 32 - Loss: 0.9860
Iteración 33 - Loss: 0.9843
Iteración 34 - Loss: 0.9893
Iteración 35 - Loss: 0.9845
Iteración 36 - Loss: 0.9882
I

Como podemos observar, modificar las "capas" de la red (`reps`), no produce ninguna diferencia. Por lo tanto, vemos que, para mejorar nuestro clasificador, modificar este parámetro no es relevante.

Vemos también que el número de iteraciones tampoco tiene ninguna relevancia, puesto que en cada iteración el loss obtenido es prácticamente el mismo, por lo que modificar ese hiperparámetro no tendría sentido.

Dada la nula evolución del loss, todo parece indicar a que el problema está en que nos encontramos en una *barren plateau*, una meseta de la cual nos es muy complicado salir, puesto que el optimizador no es capaz de encontrar una dirección en la que descienda el gradiente.

In [ ]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = NELDER_MEAD(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.

KeyboardInterrupt



KeyboardInterrupt: 

In [33]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = POWELL(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = SPSA(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.

KeyboardInterrupt



KeyboardInterrupt: 

In [ ]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = CRS(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.

KeyboardInterrupt



KeyboardInterrupt: 

In [ ]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = CRS(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.

KeyboardInterrupt



KeyboardInterrupt: 

In [ ]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = DIRECT_L(maxiter=50)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.

KeyboardInterrupt



KeyboardInterrupt: 